In [1]:
from bs4 import BeautifulSoup
from twilio.rest import Client
from datetime import datetime, timedelta
import yfinance as yf
import pandas as pd
import requests

# Application #1: Upcoming Earnings Text Updates

This Notebook contains a script that scans for company's that are releasing their earnings today. It will return a dataframe containing the tickers that are releasing their earnings. It will then send a text message to my mobile using Twilio with the formatted information regarding which companies are reporting today. 

This list of tickers will then subsequently be used to check if any of the stocks are currently consolidating using our function from the "Signals" notebook. 

### Upcoming Earnings for all US Companies

In [2]:
# initialising empty dataframe which will hold all tickers
df = pd.DataFrame()

# defining headers to simulate a browser call (helps get past some websites' defences against scraping)
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'close',
    'DNT': '1', # Do Not Track Request Header 
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}

for i in range(1, 5):
    
    # the URL which updates on each loop iteration
    url = "https://seekingalpha.com/earnings/earnings-calendar/{}".format(i)

    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url, headers=headers).text

    # get the HTML content
    soup = BeautifulSoup(html_content, "lxml")

    # obtaining tables
    table = soup.find_all('table')
    
    # indexing the relevant tables to obtain data of interest
    data = pd.read_html(str(table))[0].set_index("Release Date").iloc[:,:-1]
    
    df = df.append(data)
    
df.head(5)

,Symbol,Name,Release Time
Release Date,,,
10/07/2021,CAG,"Conagra Brands, Inc.",Pre-Market
10/07/2021,HELE,Helen of Troy Limited,Pre-Market
10/07/2021,PKE,Park Aerospace Corp.,Pre-Market
10/07/2021,TNP,Tsakos Energy Navigation Limited,Pre-Market
10/07/2021,TLRY,"Tilray, Inc.",Pre-Market


### Finding out which Companies reporting next week are consolidating now 

In [3]:
def currently_consolidating(series, perc_range):
    
    high = series.max()
    low = series.min()

    tmp = high * (100 - perc_range)/100

    if low > tmp:
        return True
    
    else:
        return False

df.index = pd.to_datetime(df.index)
today = datetime.today().strftime('%Y-%m-%d')
reporting_this_week = df[df.index < pd.to_datetime(today) + timedelta(weeks=1)]
tickers = list(reporting_this_week["Symbol"].values)

### Removing unavailable tickers from tickers list (read from file)

In [ ]:
# reading from text file and converting unavailable tickers to list 
with open ("no_data.txt", "r") as myfile:
    data = myfile.readlines()
    
unavailable_data = [d.replace(",\n", "") for d in data]

# removing said elements 
for val in unavailable_data:
    if val in tickers:
        tickers.remove(val)

### Finding consolidating, limited data, and unavailable tickers

In [4]:
unavailable_tickers = []
consolidating_tickers = []
lookback = 15
percentage = 2
count = 1

for i, ticker in enumerate(tickers):
    
    stock = yf.Ticker(ticker).history(period="{}d".format(lookback))
    
    if stock.empty:
        unavailable_tickers.append(ticker)
        
    elif len(stock) != 15:
        unavailable_tickers.append(ticker)
        
    else:
        if currently_consolidating(stock["Close"], percentage):
            print("{}: {} is currently consolidating.".format(count, ticker))
            consolidating_tickers.append(ticker)
            count += 1
        

- BDRSF: No data found for this date range, symbol may be delisted
- SVMLF: No data found, symbol may be delisted
- ZEUUF: No data found, symbol may be delisted
- QPMLF: No data found, symbol may be delisted
- HPTY: No data found, symbol may be delisted
- INLCF: No data found, symbol may be delisted
- GMNGF: No data found, symbol may be delisted
- APHD: No data found, symbol may be delisted
- BIGN: No data found for this date range, symbol may be delisted
1: CNBKA is currently consolidating.
- SYITF: No data found for this date range, symbol may be delisted
- RGRLF: No data found for this date range, symbol may be delisted
- BLIXF: No data found, symbol may be delisted
- APCDF: No data found for this date range, symbol may be delisted
- REEGF: No data found, symbol may be delisted
- MOBXF: No data found for this date range, symbol may be delisted
- PNFTF: No data found for this date range, symbol may be delisted
- IPPTF: No data found for this date range, symbol may be delisted
- NPMFF

### Updating File
- Read empty list from unavail_tickers 
- update list with unavailable tickers
- write back to file

In [60]:
# 1 opening file
f = open("no_data.txt", 'a')

# 2 set of new unavailable tickers
ut = set(unavailable_tickers)

# 3 writing unavailable tickers to file
for t in ut:
    f.write("{},".format(t))
    f.write('\n')

### Formatting the data to send a text message
- first cell: consolidating in the upcoming week
- second cell: reporting today

In [89]:
info = []

for i in range(len(consolidating_tickers)):
    
    line = df[df["Symbol"] == consolidating_tickers[i]]
    
    string = "{} ({}) is consolidating and reporting on {}. The release time is {}.".format(line["Name"].values[0], line["Symbol"].values[0], str(line.index[0])[:10], line["Release Time"].values[0])
    
    info.append(string)
    
msg1 = "\n".join(info)

In [91]:
df.index = pd.to_datetime(df.index)

today = datetime.today().strftime('%Y-%m-%d')

todays_companys = df[df.index == today]

info = []

for i in range(len(todays_companys)):
    todays_companys.iloc[i, :]
    
    string = "{} ({}) is reporting today. The release time is {}.".format(todays_companys.iloc[i, :]["Name"], todays_companys.iloc[i, :]["Symbol"], todays_companys.iloc[i, :]["Release Time"])
    
    info.append(string)
    
msg2 = "\n".join(info)

### Sending a text with Twilio

In [ ]:
# the following line needs your Twilio Account SID and Auth Token
client = Client("ACe737e611e4524cdf0ed7feb19d4ef564", "044ca567300180705e108e76715dd60e")

# change the "from_" number to your Twilio number and the "to" number
# to the phone number you signed up for Twilio with, or upgrade your
# account to send SMS to any phone number
client.messages.create(to="+447719104171", from_="+447897033826", body=msg)